In [1]:
import warnings

import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

from src.data.dataloader import MURADataset
from src.models.models import CNN, CNN_3
from netcal.metrics import ECE
from src.models.utils import pred, get_variable
from torch.autograd import Variable
from torch.nn import functional as F
import matplotlib.pyplot as plt

import cv2

import numpy as np

In [2]:
# simple implementation of CAM in PyTorch for the networks such as ResNet, DenseNet, SqueezeNet, Inception
# last update by BZ, June 30, 2021

import io
from PIL import Image
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import cv2
import json

# input image
#image_file = 'data/MURA-v1.1/valid/XR_ELBOW/patient11204/study1_negative/image1.png'
#image_file = 'data/MURA-v1.1/train/XR_FINGER/patient04887/study1_negative/image2.png'
#image_file = 'data/MURA-v1.1/valid/XR_HAND/patient11212/study1_negative/image2.png'
image_file = 'data/MURA-v1.1/valid/XR_SHOULDER/patient11723/study1_positive/image3.png'

#model_path = 'models/STATEtrained_model_epocs2_24-03-2022_14.pt'
#model_path = 'models/STATEtrained_model_epocs70_24-03-2022_22.pt'

# The model that we want to use
#model_path = 'models/STATEtrained_model_epocs100_14_04_18_trans_1.pt'

# Model on only two epochs:
model_path = 'models/STATEtrained_model_epocs2_15_04_13_trans_0.pt'

device = "cuda" if torch.cuda.is_available() else "cpu"

use_cuda = torch.cuda.is_available()


net = CNN(input_channels=3, input_height=256, input_width=256, num_classes=7).to(
    device
)

net.eval()

net.load_state_dict(
    torch.load(
        model_path,
        map_location=torch.device(device),
    )
)

net.eval()

# hook the feature extractor
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(output.data.cpu().numpy())

net._modules.get('conv5').register_forward_hook(hook_feature)

# get the softmax weight
params = list(net.parameters())
weight_softmax = np.squeeze(params[-2].data.numpy())

def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    bz, nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax.dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam


preprocess = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ]
)

# load test image
img_pil = Image.open(image_file).convert("RGB")
img_tensor = preprocess(img_pil)
img_variable = Variable(img_tensor.unsqueeze(0))
logit = net(img_variable)

# load the imagenet category list
classes = [0,1,2,3,4,5,6]


h_x = F.softmax(logit, dim=1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()

# output the prediction
for i in range(0, 5):
    print('{:.3f} -> {}'.format(probs[i], classes[idx[i]]))

# generate class activation mapping for the top1 prediction
CAMs = returnCAM(np.abs(features_blobs[0]), weight_softmax, [idx[0]])

# render the CAM and output
print('output CAM.jpg for the top1 prediction: %s'%classes[idx[0]])
img = cv2.imread(image_file)
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.3 + img * 0.5
cv2.imwrite('reports/CAM.jpg', result)



torch.Size([1, 7, 32, 32])
torch.Size([1, 7])
0.170 -> 4
0.169 -> 0
0.152 -> 5
0.132 -> 1
0.132 -> 3
output CAM.jpg for the top1 prediction: 4


True

In [116]:
net._modules.get(finalconv_name)

Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))

In [117]:
features_blobs[0].shape

(1, 64, 32, 32)

In [114]:
net

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (dropout): Dropout2d(p=0.2, inplace=False)
  (l_out): Linear(in_features=16384, out_features=7, bias=False)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [121]:
net

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(2, 2), padding=(2, 2))
  (conv4): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (dropout): Dropout2d(p=0.2, inplace=False)
  (l_out): Linear(in_features=16384, out_features=7, bias=False)
  (maxpool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=

In [127]:
import torch
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self, img_size, num_class):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            # 3 x ? x ?
            nn.Conv2d(3, 32, 3, 1, 1),
            nn.LeakyReLU(0.2),
            # 32 x ? x ?
            nn.Conv2d(32, 64, 3, 1, 1),
            nn.LeakyReLU(0.2),
            # 64 x ? x ?
            nn.MaxPool2d(2, 2),
            # 64 x ? / 2 x ? / 2
            nn.Conv2d(64, 128, 3, 1, 1),
            nn.LeakyReLU(0.2),
            # 128 x ? / 2 x ? / 2
            nn.Conv2d(128, 256, 3, 1, 1),
            nn.LeakyReLU(0.2),
            # 256 x ? / 2 x ? / 2
            nn.MaxPool2d(2, 2),
            # 256 x ? / 4 x ? / 4
            nn.Conv2d(256, 512, 3, 1, 1),
            nn.LeakyReLU(0.2),
            # 512 x ? / 4 x ? / 4
            nn.MaxPool2d(2, 2),
            # 512 x ? / 8 x ? / 8
            nn.Conv2d(512, num_class, 3, 1, 1),
            nn.LeakyReLU(0.2)
        )
        self.avg_pool = nn.AvgPool2d(img_size // 8)
        self.classifier = nn.Linear(num_class, num_class)

    def forward(self, x):
        features = self.conv(x)
        flatten = self.avg_pool(features).view(features.size(0), -1)
        output = self.classifier(flatten)
        return output, features

In [134]:
net = CNN(128, 100)

In [135]:
net

CNN(
  (conv): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): LeakyReLU(negative_slope=0.2)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): LeakyReLU(negative_slope=0.2)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LeakyReLU(negative_slope=0.2)
    (7): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): LeakyReLU(negative_slope=0.2)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): LeakyReLU(negative_slope=0.2)
    (12): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (13): Conv2d(512, 100, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): LeakyReLU(negative_slope=0.2)
  )
  (avg_pool): AvgPool2